Subset_finding is a modul in shapiq_student which allows to compute the set of x features of an InteractionValues object which have the highest or lower shapley value. The following Jupyter Notebook explains the different algorithms used to determine the set and showcases how to use them.

In [ ]:
import sys, os

sys.path.append(os.path.abspath(".."))
import shapiq_student.coalition_finding as cf

Importing the subset_finding module.

In [ ]:
from shapiq import ExactComputer
from shapiq.games.benchmark import SOUM

Importing modules from shapiq used to generate InteractionValues to showcase the subset_finding module on.

In [ ]:
game = SOUM(n=5, n_basis_games=100)
computer = ExactComputer(n_players=game.n_players, game=game)
Interaction_Values = computer(index="FSII", order=3)

This generates InteractionValues using functions from shapiq.

In [ ]:
Interaction_Values.dict_values

{(): 0.8188664227426776,
 (0,): 0.1640937022012522,
 (1,): -0.40398748802120926,
 (2,): -0.5056002071191349,
 (3,): -2.8614891699074887,
 (4,): -0.029946596700750432,
 (0, 1): -0.6315173070901232,
 (0, 2): -1.1349814390343986,
 (0, 3): -0.3020736331570433,
 (0, 4): -0.3269094226237127,
 (1, 2): -0.1992231439062453,
 (1, 3): -1.215209124121748,
 (1, 4): -0.3761106695116966,
 (2, 3): -0.5200689548248226,
 (2, 4): -1.176454989554101,
 (3, 4): -0.7249340770501714,
 (0, 1, 2): 0.017345413078240685,
 (0, 1, 3): -0.38551689784130483,
 (0, 1, 4): 0.5295082184972789,
 (0, 2, 3): -0.16281549970094236,
 (0, 2, 4): 0.5295082185056167,
 (0, 3, 4): 0.01636871094341243,
 (1, 2, 3): 0.584371912275799,
 (1, 2, 4): 0.6224009337095094,
 (1, 3, 4): -0.3843970941726946,
 (2, 3, 4): 1.7614543973266574}

This is an example for the Shapley Values in an InteractionValues object with 5 features. 

To determine the coalition of x features with the largest or smallest combined value multiple different algorithms can be used. In this example our coalition will include 3 features.

In [ ]:
brute_force_output = cf.brute_force(Interaction_Values, 3)

A simple but slow way is to use a brute force approach to check every possible coalition and return the smallest and largest.

In [ ]:
print(brute_force_output)

InteractionValues(
    index=FSII, max_order=3, min_order=0, estimated=True, estimation_budget=None,
    n_players=5, baseline_value=0.8188664227426776,
    Top 10 interactions:
        (): 0.8188664227426776
        (0, 1, 4): -0.2560031405062835
        (1, 3, 4): -5.177207796743081
)


This way the output is guranteed to be correct at the downside of significant runtime.

In the subset_finding module there are three more algorithms implemented, each with their different accuracy and runtime.
Every function is called with the InteractionValues object and the number of features in the coalition.

## Greedy Coalition Method

The `greedy_coalition` method selects features one-by-one by always choosing the one that locally improves the coalition value the most (or the least for minimization). This is a fast heuristic method.

Below, we demonstrate its usage:

In [ ]:
greedy_output = cf.greedy_coalition(Interaction_Values, 3)
print(greedy_output)

InteractionValues(
    index=FSII, max_order=3, min_order=0, estimated=True, estimation_budget=None,
    n_players=5, baseline_value=0.8188664227426776,
    Top 10 interactions:
        (): 0.8188664227426776
        (0, 1, 4): -0.2560031405062835
        (1, 3, 4): -5.177207796743081
)


## Beam Search Coalition Method

The `beam_search_coalition` method performs a breadth-limited search through possible coalitions. At each step, it retains only the top `beam_width` best partial coalitions, which makes it more robust than greedy but still efficient.

Below, we demonstrate its usage:

In [ ]:
beam_search_output = cf.beam_search_coalition(Interaction_Values, 3)
print(beam_search_output)

InteractionValues(
    index=FSII, max_order=3, min_order=0, estimated=True, estimation_budget=None,
    n_players=5, baseline_value=0.8188664227426776,
    Top 10 interactions:
        (): 0.8188664227426776
        (0, 1, 4): -0.2560031405062835
        (1, 3, 4): -5.177207796743081
)


## Beam Search Coalition Method with specified beam_width

To customize the beam_width, one can use '`beam_search_coalition_call`' as demonstrated below:

In [ ]:
beam_search_call_output = cf.beam_search_coalition_call(Interaction_Values, 3, beam_width=10)
print(beam_search_call_output)

InteractionValues(
    index=FSII, max_order=3, min_order=0, estimated=True, estimation_budget=None,
    n_players=5, baseline_value=-1.9944612118945846,
    Top 10 interactions:
        (1, 3, 4): -1.2029500754229137
        (): -1.9944612118945846
        (0, 2, 4): -5.42441447603113
)


## Recursive greedy algorithm

The recursive greedy algorithm splits the features into candidates for the maximum and minimum coalition based on their first order Shapley Value. Then for every candidate is recursively computes the best candidates to be in the coalition with them based on their shared Shapley Values. For every potential member added to the coalition a smaller fraction of next candidates is checked until for the last member of the coalition only the best candidate is selected.

In [ ]:
recursive_greedy_output = cf.recursive_greedy_coalition(Interaction_Values, 3)

In [ ]:
print(recursive_greedy_output)

InteractionValues(
    index=FSII, max_order=3, min_order=0, estimated=True, estimation_budget=None,
    n_players=5, baseline_value=-1.0798937315685124,
    Top 10 interactions:
        (3, 2, 4): -0.6822053214857471
        (): -1.0798937315685124
        (1, 2, 0): -5.466691693392462
)


An advantage of the recursive greedy algorithm is that the maximum and minimum coalition has to be computed seperatly which means that either can be accessed on its own in less runtime if only one is needed.

In [ ]:
recursive_greedy_min_output = cf.recursive_greedy_min_coalition(Interaction_Values, 3)

In [ ]:
print(recursive_greedy_min_output)

InteractionValues(
    index=FSII, max_order=3, min_order=0, estimated=True, estimation_budget=None,
    n_players=5, baseline_value=-1.0798937315685124,
    Top 10 interactions:
        (): -1.0798937315685124
        (1, 2, 0): -5.466691693392462
)


In [ ]:
recursive_greedy_max_output = cf.recursive_greedy_max_coalition(Interaction_Values, 3)

In [ ]:
print(recursive_greedy_max_output)

InteractionValues(
    index=FSII, max_order=3, min_order=0, estimated=True, estimation_budget=None,
    n_players=5, baseline_value=-1.0798937315685124,
    Top 10 interactions:
        (3, 2, 4): -0.6822053214857471
        (): -1.0798937315685124
)
